In [119]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import warnings
import lightgbm as lgb
from workflow import main_indicators,Larry_simples
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [195]:
ativo = 'wdo_1_full'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)

In [196]:
df_ = main_indicators(df)

#df_acao = Larry_simples(df_new, tempo_fechar=-20)

In [242]:
tempo_fechar = -10
df['MA_9'] = ta.EMA(df['Close'], timeperiod = 9)
df['MA_9_diff_value'] = df['MA_9'].diff(periods = 1)
df['MA_9_tend'] = df['MA_9_diff_value'].apply(lambda x: 1 if x > 0 else -1)
df['MA_9_tend_diff'] = df['MA_9_tend'].diff()

df['acao'] = df['MA_9_tend_diff'].apply(lambda x: 'call' if x == 2 else 0)
df['preco'] = df.apply(lambda x: x['High'] if x['acao']=='call' else x['Low'] if x['acao'] == 'sell' else 0, axis =1)
df['stop'] = df.apply(lambda x: x['Low'] if x['acao']=='call' else x['High'] if x['acao'] == 'sell' else 0, axis =1)

df['realizacao_simplista'] = df['Close'].shift(tempo_fechar)

df = df.dropna()

In [243]:
lista = df['acao'].values

In [244]:
tempo_fechar = 10
for num, i in enumerate(lista):
    #print(num,i)
    if i != 0:
        for i in range(tempo_fechar):
            try:
                n_num = num + i + 1
                lista[n_num] = 0
            except:
                pass

In [245]:
df['acao'] = lista

df_acao = df[df['acao'] != 0]



def cal_result(x):
    if x['acao'] == 'call':
        if x['realizacao_simplista'] < x['stop']:
            resultado = x['stop'] - x['preco']
            return resultado
        else:
            resultado = x['realizacao_simplista'] - x['preco']
            return resultado
    if x['acao'] == 'sell':
        if x['realizacao_simplista'] > x['stop']:
            resultado = x['stop'] - x['preco']
            return resultado
        else:
            resultado = x['realizacao_simplista'] - x['preco']
            return resultado

df_acao['resultado_valor'] = df_acao.apply(cal_result, axis = 1)
df_acao['resultado_binario'] = df_acao['resultado_valor'].apply(lambda x: 1 if x > 0 else 0)


In [248]:
df_acao.head(50)

,Data,Open,High,Low,Close,Vol,MA_5,MA_21,SAR,BB_up,...,MA_9,MA_9_diff_value,MA_9_tend,MA_9_tend_diff,acao,preco,stop,realizacao_simplista,resultado_valor,resultado_binario
116,2022-03-31 16:26:00,4793.0,4795.5,4793.0,4795.5,4243,4793.4,4792.761905,4769.0,4795.197235,...,4793.588889,0.477778,1,2.0,call,4795.5,4793.0,4798.0,2.5,1
127,2022-03-31 16:37:00,4798.0,4799.0,4797.5,4799.0,2213,4798.9,4796.476190,4769.0,4803.034953,...,4798.492250,0.126938,1,2.0,call,4799.0,4797.5,4795.5,-1.5,0
142,2022-03-31 16:52:00,4795.5,4796.5,4795.0,4796.0,1577,4795.2,4796.690476,4769.0,4799.724100,...,4795.649037,0.087741,1,2.0,call,4796.5,4795.0,4795.5,-1.0,0
159,2022-03-31 17:09:00,4795.5,4797.0,4795.0,4796.5,2420,4795.7,4796.857143,4769.0,4800.265117,...,4796.191444,0.077139,1,2.0,call,4797.0,4795.0,4790.5,-2.0,0
188,2022-03-31 17:38:00,4785.5,4785.5,4784.5,4785.5,1134,4785.1,4786.571429,4769.0,4790.016544,...,4785.308807,0.047798,1,2.0,call,4785.5,4784.5,4782.5,-1.0,0
206,2022-03-31 17:56:00,4780.0,4781.5,4778.0,4781.0,1151,4779.8,4781.714286,4769.0,4785.434585,...,4780.333951,0.166512,1,2.0,call,4781.5,4778.0,4763.5,-3.5,0
239,2022-04-01 09:29:00,4752.0,4754.5,4750.0,4753.5,6834,4751.3,4756.595238,4803.5,4765.029856,...,4752.881644,0.154589,1,2.0,call,4754.5,4750.0,4728.0,-4.5,0
254,2022-04-01 09:44:00,4733.5,4738.5,4733.0,4737.5,9338,4732.9,4741.738095,4803.5,4757.594443,...,4735.376332,0.530917,1,2.0,call,4738.5,4733.0,4733.0,-5.5,0
272,2022-04-01 10:02:00,4732.0,4741.5,4731.5,4740.5,22718,4734.0,4734.000000,4803.5,4739.465674,...,4734.094206,1.601449,1,2.0,call,4741.5,4731.5,4734.5,-7.0,0
283,2022-04-01 10:13:00,4735.0,4738.5,4733.5,4738.0,8527,4737.3,4734.619048,4803.5,4739.886891,...,4736.168220,0.457945,1,2.0,call,4738.5,4733.5,4739.5,1.0,1


In [247]:
compra = df_acao[df_acao['acao'] =='call']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='call'].shape[0]
venda = df_acao[df_acao['acao'] =='sell']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='sell'].shape[0]

compra_result = df_acao[df_acao['acao'] =='call']['resultado_valor'].sum()
venda_result = df_acao[df_acao['acao'] =='sell']['resultado_valor'].sum()

print(f'Venda: {venda}% -- Compra: {compra}%')
print(f'Venda: {venda_result} -- Compra: {compra_result}')

Venda: nan% -- Compra: 0.4233670653173873%
Venda: 0.0 -- Compra: 3710.5


In [249]:
df_new = df_acao.drop(columns = ['MA_9_diff_value', 'MA_9_tend', 'MA_9_tend_diff', 'preco','stop', 'realizacao_simplista'])
df_new = df_new.set_index(['Data','resultado_valor'])

In [250]:
df_new_sell = df_new[df_new['acao'] == 'call']
df_new_sell = df_new_sell.drop(columns = ['acao'])

In [251]:
df_new_sell = df_new.drop(columns = ['acao'])

In [252]:
X = df_new_sell.drop(columns = 'resultado_binario')
y = df_new_sell['resultado_binario']

In [253]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [254]:
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

LGBMClassifier()

In [255]:
# predict the results
y_pred=clf.predict(X_test)

In [256]:
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

LightGBM Model accuracy score: 0.5346


In [257]:
print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))

Training set score: 0.9435
Test set score: 0.5346


In [258]:
X_test['new'] = y_pred
a = X_test.reset_index()

In [259]:
a[a['new'] == 1]['resultado_valor'].sum()

346.5

In [306]:
import pycaret

ModuleNotFoundError: No module named 'pycaret'